# EXPLOTATION 04 _ 영화 리뷰 텍스트 감성 분석하기

## 이 노드의 루브릭

   1. 다양한 방법됨으로 Text Classification 태스크를 성공적으로 구현하였다.     
   > "3가지 이상의 모델이 성공적으로 시도됨"     
   
   2. gensim을 활용하여 자체 학습된/사전 학습된 임베딩 레이어를 분석하였다.      
   > "gensim의 유사단어 찾기를 활용하여 자체학습한 임베딩과 사전학습 임베딩을 적절히 분석함" 
   
   3. 한국어 Word2Vec을 이용하여 가시적인 성능 향상을 달성하였다.      
   > "네이버 영화 리뷰 데이터 감성분석의 정확도를 85% 이상 달성하였다"

위의 루브릭을 참고하여, 네이버 영화리뷰 감성분석 실습을 진행해본다.

### 목차     

1. 데이터 준비와 확인     
2. 데이터 로더 구성
    - 데이터 중복 제거, NaN 결측치 제거, 한국어 토크나이저로 토큰화, 불용어 제거, 사전 word_to_index등을 구성하는 데이터 로더 함수 구현      


3. 모델 구성을 위한 데이터 분석 및 가공     
4. 모델구성 및 validation set 구성
    - validation set 구성
    - 1-D CNN    
    - RNN   
    - GlobalMaxPooling1D    
    
5. 모델 훈련 개시
6. Loss, Accuracy 그래프 시각화
7. 학습된 Embedding 레이어 분석
8. 한국어 Word2Vec 활용하여 성능 개선

## 1. 데이터 준비와 확인

In [16]:
import pandas as pd
import urllib.request 

%matplotlib inline
import matplotlib.pyplot as plt 
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

train_data = pd.read_table('~/SUBMIT_MISSION_GIT/ex4_Text/naver/ratings_train.txt')
test_data = pd.read_table('~/SUBMIT_MISSION_GIT/ex4_Text/naver/ratings_test.txt')

train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


> 무사히 데이터를 불러왔음을 확인할 수 있다. 

## 2. 데이터 로더 구성     

2-1. 데이터 로더를 구성하기 전에, 루브릭에서 요구하는 사항의 의미를 각각 확인해본다.   
* 데이터의 중복 제거   
* NaN 결측치 제거     
    - 결측치란? : 결측치는 값이 빈 것을 말하는데, 주로 삭제하거나 대체하는 것으로 처리한다고 한다.    
* 한국어 토크나이저로 토큰화    
    - 토큰화? : 구조화 된 텍스트 집단(코퍼스)에서 단위로 나누는 작업(ex)의미단위로 토큰화)    
    - 토큰화를 할 때는 일반적으로 단순 띄어쓰기나 문장부호 단위로 자르지 않도록 조심하는데, 우리는 토큰화를 할 때 띄어쓰기 단위로 진행하는 것을 생각해 볼 수 있겠다.    
    - [토큰화는 여기를 참고](https://wikidocs.net/21698)      
* 불용어 제거    
    - 불용어 : 자주 등장하지만 의미분석에 큰 영향을 주지 않는 나, 너, 조사 등을 의미     
* 딕셔너리 word_to_index(단어를 벡터로 표현) 구성    
* 텍스트 스트링을 사전 인덱스 스트링으로 변환   
* x_train, y_train, x_test, y_test, word_to_index 리턴

In [ ]:
from konlpy.tag import Mecab
tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=num_words):
    
    
x_train, y_train, x_test, y_test, word_to_index = load_data(train_data, test_data)

In [ ]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

In [ ]:
index_to_word = {index:word for word, index in word_to_index.items()}